In [1]:
import pandas as pd
from mistralai import Mistral
from dotenv import load_dotenv
import os
import asyncio
load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")  # your API key
questions_file = '../data/dataset/questions.csv'  # path to the questions file

output_path = '../data/output/'  # path to the output file

df = pd.read_csv(questions_file)

question_prompt = lambda body, possible_answer_a, possible_answer_b, possible_answer_c, possible_answer_d, possible_answer_e: (
    "Answer the following question with the letters of the correct answer. Each question can have multiple answers that are right. "
    "For each answwer think this way: 'is it true compared to the question provided?'"
    "For each question first explain the medical condition and what are the implication, give context, then print the corrects answers."
    "Your answer must contain the letter of the answers, separated by commas and without any space. An ideal output is like: 'A,B', for instance."
    "You output the letters that you are asked to provide, e.g. 'A,B,C' or 'C'. Your answer is always sorted alphabetically. You must not put letters"
    "in a different order"
    f"{body}\n"
    f"A: {possible_answer_a}\n"
    f"B: {possible_answer_b}\n"
    f"C: {possible_answer_c}\n"
    f"D: {possible_answer_d}\n"
    f"E: {possible_answer_e}\n"
)

answers = []
client = Mistral(api_key=api_key)

prompts = [question_prompt(
      row["question"],
      row["answer_A"],
      row["answer_B"],
      row["answer_C"],
      row["answer_D"],
      row["answer_E"]) for row_idx, row in df.iterrows()]


In [7]:
prompts

["Answer the following question with the letters of the correct answer. Each question can have multiple answers that are right. For each answwer think this way: 'is it true compared to the question provided?'For each question first explain the medical condition and what are the implication, give context, then print the corrects answers.Your answer must contain the letter of the answers, separated by commas and without any space. An ideal output is like: 'A,B', for instance.You output the letters that you are asked to provide, e.g. 'A,B,C' or 'C'. Your answer is always sorted alphabetically. You must not put lettersin a different orderDevant un exanthème roséoliforme fébrile de l'enfant, les principales étiologies sont (une ou plusieurs réponsesexactes):\nA: Un exanthème subit\nB: Un mégalérythème épidémique\nC: Une rubéole\nD: Une mononucléose infectieuse\nE: Un syndrome de kawasaki\n",
 "Answer the following question with the letters of the correct answer. Each question can have multi

In [11]:
async def get_async_response():
    chat_response = await client.chat.complete_async(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": prompt} for prompt in prompts],
        temperature=0.0,
    )
    if chat_response is not None:
        answers.append(chat_response.choices[0].message.content)

    # output format is a 2-columns dataframe with exactly 103 rows
    output_df = pd.DataFrame(answers, columns=["Answer"])
    output_df.index.name = "id"

    os.makedirs(output_path, exist_ok=True)

    output_df.to_csv(f"{output_path}output_verbose.csv")

In [12]:
asyncio.run(get_async_response())

RuntimeError: asyncio.run() cannot be called from a running event loop